In [ ]:
# pip install neat-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.3 MB/s eta 0:00:00


In [1]:
from torchvision import datasets, transforms
import torch
import neat
# TODO: These are a beginning of an own implemenation of NEAT, not used anywhere yet.
class Genome:
    def __init__(self, key):
        self.key = key

class NEAT:
    def __init__(self, inputNodes, outputNodes):
        self.inputLayer =  torch.nn.Linear(in_features= inputNodes,out_features=1)
        self.relu1 = torch.nn.ReLU()
        self.outputLayer = torch.nn.Linear(1,outputNodes)
        self.model = [self.inputLayer, self.relu1,self.outputLayer]
        self.nLayers = 1


class Model:
    def __init__(self,inputNodes,outputNodes):
        self.inputNodes = inputNodes
        self.outputNodes = outputNodes


/home/nils/Documents/naturliga_beräkningsmetoder/project/NEAT-PSO/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/nils/Documents/naturliga_beräkningsmetoder/project/NEAT-PSO/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [3]:
from torchvision import datasets, transforms
import torch

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

dataset = datasets.MNIST(root='../data', train=True, download=True, transform=transform);
trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True);


In [4]:
train_dataset = dataset.data
train_dataset = train_dataset.view(-1,784)[:10]
train_labels = dataset.targets
train_labels = torch.eye(10)[train_labels[:10]]


In [16]:
train_labels[0]

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [5]:
print(train_dataset)
print("================================")
print(train_labels)
train_dataset.view(-1,784)[:10].shape

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.uint8)
tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])


torch.Size([10, 784])

In [69]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        # print(f"Genome ID: {genome_id} : {genome}")
        fitness = []
        for xi, xo in zip(train_dataset, train_labels):
            # print(xi.shape)
            output = torch.tensor(net.activate(xi))
            fitness.append(- float(torch.nn.functional.cross_entropy(output , xo)))
            # print(genome.fitness)
        genome.fitness = sum(fitness) / len(fitness)
        

In [68]:
torch.nn.functional.cross_entropy(torch.tensor([1.0,0,0,0,1]),torch.tensor([1.0,0,0,0,1]))

t1 = torch.tensor([1.0,0,0,0,1]) 

t2 = torch.tensor([1.0,0,0,0,1])

((t1 - t2)**2).sum()

torch.nn.functional.binary_cross_entropy(t1,t2)

tensor(0.)

In [72]:
"""
Runs evaluation functions in parallel subprocesses
in order to evaluate multiple genomes at once.
"""
from multiprocessing import Pool


class ParallelEvaluator(object):
    def __init__(self, num_workers, eval_function, timeout=None, maxtasksperchild=None):
        """
        eval_function should take one argument, a tuple of (genome object, config object),
        and return a single float (the genome's fitness).
        """
        self.eval_function = eval_function
        self.timeout = timeout
        self.pool = Pool(processes=num_workers, maxtasksperchild=maxtasksperchild)

    def __del__(self):
        self.pool.close()
        self.pool.join()
        self.pool.terminate()


    def evaluate(self, genomes, config):
        jobs = []
        for ignored_genome_id, genome in genomes:
            jobs.append(self.pool.apply_async(self.eval_function, (genome, config)))

        # assign the fitness back to each genome
        for job, (ignored_genome_id, genome) in zip(jobs, genomes):
            genome.fitness = job.get(timeout=self.timeout)

In [70]:
def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))
    print("before winner")
    # Run for up to 300 generations.
    parallel_eval = ParallelEvaluator(8,eval_genomes)
    winner = p.run(parallel_eval, 10)
    print("after winner")
    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

    #Evaluation
    acc_counter = []
    for xi, xo in zip(train_dataset, train_labels):
        print("inside foor loop")
        xi = xi.flatten()
        output = winner_net.activate(xi)
        print("target output {!r}, actual output {!r}".format(xo, output))
        print(f"target output {torch.argmax(xo)}, actual output {torch.argmax(torch.tensor(output))}")
        print("==================================")
        acc_counter.append(1.0 if torch.argmax(xo) ==torch.argmax(torch.tensor(output)) else 0.0)
    print(f"Overall Accuracy: {torch.tensor(acc_counter).mean().item()}")
    return winner_net, winner;

In [71]:
winner_1, winner_genome = run('NEAT/config-feedforward');

	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_conn

before winner

 ****** Running generation 0 ****** 

Population's average fitness: -2.45061 stdev: 0.09326
Best fitness: -2.28247 - size: (20, 7940) - species 2 - id 2
Average adjusted fitness: 0.161
Mean genetic distance 3.015, standard deviation 0.906
Population of 20 members in 10 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     2     -2.4    0.177     0
     2    0     2     -2.3    0.329     0
     3    0     2     -2.3    0.280     0
     4    0     2     -2.5    0.075     0
     5    0     2     -2.5    0.129     0
     6    0     2     -2.4    0.163     0
     7    0     2     -2.6    0.000     0
     8    0     2     -2.4    0.216     0
     9    0     2     -2.5    0.079     0
    10    0     2     -2.5    0.161     0
Total extinctions: 0
Generation time: 35.244 sec

 ****** Running generation 1 ****** 

Population's average fitness: -2.45760 stdev: 0.10791
Best fitness: -2.16262 - size: (21, 7908) - species 2 - id 2

In [65]:
n = (winner_1.create(winner_genome,neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         'config-feedforward')))

In [68]:
n.input_nodes

[-1,
 -2,
 -3,
 -4,
 -5,
 -6,
 -7,
 -8,
 -9,
 -10,
 -11,
 -12,
 -13,
 -14,
 -15,
 -16,
 -17,
 -18,
 -19,
 -20,
 -21,
 -22,
 -23,
 -24,
 -25,
 -26,
 -27,
 -28,
 -29,
 -30,
 -31,
 -32,
 -33,
 -34,
 -35,
 -36,
 -37,
 -38,
 -39,
 -40,
 -41,
 -42,
 -43,
 -44,
 -45,
 -46,
 -47,
 -48,
 -49,
 -50,
 -51,
 -52,
 -53,
 -54,
 -55,
 -56,
 -57,
 -58,
 -59,
 -60,
 -61,
 -62,
 -63,
 -64,
 -65,
 -66,
 -67,
 -68,
 -69,
 -70,
 -71,
 -72,
 -73,
 -74,
 -75,
 -76,
 -77,
 -78,
 -79,
 -80,
 -81,
 -82,
 -83,
 -84,
 -85,
 -86,
 -87,
 -88,
 -89,
 -90,
 -91,
 -92,
 -93,
 -94,
 -95,
 -96,
 -97,
 -98,
 -99,
 -100,
 -101,
 -102,
 -103,
 -104,
 -105,
 -106,
 -107,
 -108,
 -109,
 -110,
 -111,
 -112,
 -113,
 -114,
 -115,
 -116,
 -117,
 -118,
 -119,
 -120,
 -121,
 -122,
 -123,
 -124,
 -125,
 -126,
 -127,
 -128,
 -129,
 -130,
 -131,
 -132,
 -133,
 -134,
 -135,
 -136,
 -137,
 -138,
 -139,
 -140,
 -141,
 -142,
 -143,
 -144,
 -145,
 -146,
 -147,
 -148,
 -149,
 -150,
 -151,
 -152,
 -153,
 -154,
 -155,
 -156,
 -157,
 -158,
 -